In [1]:
import sys
import os
sys.path = [os.path.abspath(os.path.join(os.curdir,'..'))] + sys.path

In [2]:
# from tiledb_xarray import TileDBDataSetBuilder
import netCDF4
import h5py
import json
import os

In [3]:
import importlib 
import tiledb_xarray
import tiledb_xarray.hdf5_tiledb_writer 
importlib.reload(tiledb_xarray)
importlib.reload(tiledb_xarray.hdf5_tiledb_writer)
from  tiledb_xarray import get_data_datasets_and_others
from tiledb_xarray.hdf5_tiledb_writer import TileDBDataSetBuilderDimsInAttrs,HDF5AttrsEncoder, DIM_KEY

In [4]:
DATA_DIR = os.path.abspath(os.path.join(os.curdir,'..', 'data'))
in_file = os.path.join(DATA_DIR, "apepda.pc51150.nc")

In [9]:
# open_file.close()

In [5]:
open_file = h5py.File(in_file,'r')


## Finding just data arrays by excluding bnds, dims, etc

In [42]:

data_ds, other_ds = get_data_datasets_and_others(open_file)

print("Data:")
display(data_ds)

print("Other:")
display(other_ds)

Data:


{'air_pressure_at_sea_level',
 'air_temperature',
 'cloud_area_fraction',
 'relative_humidity',
 'specific_humidity',
 'surface_air_pressure',
 'surface_downwelling_longwave_flux_in_air',
 'surface_downwelling_shortwave_flux_in_air'}

Other:


{'bnds',
 'forecast_period',
 'forecast_period_0',
 'forecast_period_0_bnds',
 'forecast_reference_time',
 'grid_latitude',
 'grid_longitude',
 'height',
 'rotated_latitude_longitude',
 'time',
 'time_0',
 'time_0_bnds'}

## find shared domains

In [32]:
exclude = set()
all_items = set()
open_file.visititems(partial(excluder, exclude))
open_file.visit(lambda x: all_items.add(x))



data_sets = all_items - exclude


domains = {}
for ds_name in data_sets:
    ds = open_file[ds_name]
    key = json.dumps(ds.attrs[DIM_KEY], default=HDF5AttrsEncoder(ds.file).default) + "__" + str(ds.shape)
    dom = domains.get(key,[])
    dom.append(ds_name)
    domains[key] = dom

domains



{'[["/time_0"], ["/grid_latitude"], ["/grid_longitude"]]__(32, 219, 286)': ['cloud_area_fraction',
  'surface_downwelling_longwave_flux_in_air',
  'surface_downwelling_shortwave_flux_in_air'],
 '[["/time"], ["/grid_latitude"], ["/grid_longitude"]]__(33, 219, 286)': ['relative_humidity',
  'specific_humidity',
  'air_pressure_at_sea_level',
  'air_temperature',
  'surface_air_pressure']}

## Build ds with Multi phonomeno 

In [42]:


importlib.reload(tiledb_xarray)
importlib.reload(tiledb_xarray.hdf5_tiledb_writer)
from tiledb_xarray.hdf5_tiledb_writer import  MultiPhonomBuilder

In [43]:
import numpy as np
diffs = np.diff([1,2,3,4,5])
all([diffs[i] == diffs[i-1] for i in range(1, len(diffs))])

True

In [41]:

open_file['time_0'].name

'/time_0'

In [51]:
for i, val in enumerate(open_file['time_0'][()]):
    print(f"{3*i + -1043134.5 == val}")

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [44]:
out = DATA_DIR + '/multi_phonom'
!rm -rf {out} 
builder = MultiPhonomBuilder(out, open_file, dynamic_coords=['time', 'time_0'])
builder.build()

group <HDF5 file "apepda.pc51150.nc" (mode r)> type <class 'h5py._hl.files.File'>
group name /
made_group / at /Users/theo/repos/crd_tiledb_xarray/data/multi_phonom/
wrote group attrs for /
create multi for [[time_0][grid_latitude][grid_longitude]] at /Users/theo/repos/crd_tiledb_xarray/data/multi_phonom/[[time_0][grid_latitude][grid_longitude]]
time_0 ['time', 'time_0']
time_0
is dynamic
coord time_0 == 3.0 x i + -1043134.5
grid_latitude ['time', 'time_0']
grid_latitude
grid_longitude ['time', 'time_0']
grid_longitude


TileDBError: [TileDB::ArraySchema] Error: Array schema check failed; Attributes and dimensions must have unique names

In [10]:
A[-10:-1,1,1]
A.close()

In [11]:
import tiledb
A =  tiledb.open('/Users/theo/repos/crd_tiledb_xarray/data/multi_phonom/[[time][grid_latitude][grid_longitude]]','r')
display(A.nonempty_domain())
print(A[0:3,0:1,2:3])
print(A[-10:-1,1,1])
A.close()

((0, 32), (0, 218), (0, 285))

OrderedDict([('relative_humidity', array([[[100.]],

       [[100.]],

       [[100.]]], dtype=float32)), ('air_temperature', array([[[263.58997]],

       [[261.53253]],

       [[260.20972]]], dtype=float32)), ('surface_air_pressure', array([[[100834.]],

       [[100956.]],

       [[101078.]]], dtype=float32)), ('air_pressure_at_sea_level', array([[[101628.]],

       [[101752.]],

       [[101877.]]], dtype=float32)), ('specific_humidity', array([[[0.00167337]],

       [[0.00139362]],

       [[0.00123442]]], dtype=float32))])
OrderedDict([('relative_humidity', array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)), ('air_temperature', array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)), ('surface_air_pressure', array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)), ('air_pressure_at_sea_level', array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)), ('specific_humidity', array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32))])


## Build a DS with the TileDBDataSetBuilderDimsInAttrs

In [ ]:
data_sets = ds - exclue
data_sets

In [ ]:
MY_DS = "specific_humidity"
ds = open_file[MY_DS]

In [ ]:
ds.file

In [70]:
import tiledb_xarray.hdf5_tiledb_writer

importlib.reload(tiledb_xarray.hdf5_tiledb_writer)
from  tiledb_xarray.hdf5_tiledb_writer import HDF5DSEncoder
from tiledb_xarray.hdf5_tiledb_writer import TileDBDataSetBuilderDimsInAttrs,HDF5AttrsEncoder

In [71]:
OUT = DATA_DIR+"/new-tiledb"
! rm -r {OUT}
builder = TileDBDataSetBuilderDimsInAttrs(OUT, ds.file)
builder.build()
builder.close()

Exclude: {'height', 'time_0_bnds', 'forecast_period', 'time', 'forecast_reference_time', 'grid_longitude', 'time_0', 'grid_latitude', 'forecast_period_0', 'forecast_period_0_bnds', 'bnds'}
group <HDF5 file "apepda.pc51150.nc" (mode r)> type <class 'h5py._hl.files.File'>
group name /
made_group / at /Users/theo/repos/crd_tiledb_xarray/data/new-tiledb/
wrote group attrs for /
air_pressure_at_sea_level not in {'height', 'time_0_bnds', 'forecast_period', 'time', 'forecast_reference_time', 'grid_longitude', 'time_0', 'grid_latitude', 'forecast_period_0', 'forecast_period_0_bnds', 'bnds'}
make ds air_pressure_at_sea_level in group None
air_temperature not in {'height', 'time_0_bnds', 'forecast_period', 'time', 'forecast_reference_time', 'grid_longitude', 'time_0', 'grid_latitude', 'forecast_period_0', 'forecast_period_0_bnds', 'bnds'}
make ds air_temperature in group None
cloud_area_fraction not in {'height', 'time_0_bnds', 'forecast_period', 'time', 'forecast_reference_time', 'grid_longitud

In [ ]:
import tiledb

In [ ]:
A = tiledb.open(OUT+'/cloud_area_fraction','r')

In [ ]:
A.nonempty_domain()

In [ ]:
{k:json.loads(v) for k,v in A.meta.items()}


## Try group datasets into domains


## NetCDF lib

In [6]:
ncds = netCDF4.Dataset(in_file)
ncds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    source: Data from Met Office Unified Model
    Conventions: CF-1.5
    dimensions(sizes): time(33), grid_latitude(219), grid_longitude(286), time_0(32), bnds(2)
    variables(dimensions): float32 air_pressure_at_sea_level(time,grid_latitude,grid_longitude), int32 rotated_latitude_longitude(), float64 time(time), float32 grid_latitude(grid_latitude), float32 grid_longitude(grid_longitude), float64 forecast_period(time), float64 forecast_reference_time(), float32 air_temperature(time,grid_latitude,grid_longitude), float64 height(), float32 cloud_area_fraction(time_0,grid_latitude,grid_longitude), float64 time_0(time_0), float64 time_0_bnds(time_0,bnds), float64 forecast_period_0(time_0), float64 forecast_period_0_bnds(time_0,bnds), float32 relative_humidity(time,grid_latitude,grid_longitude), float32 specific_humidity(time,grid_latitude,grid_longitude), float32 surface_air_pressure(time,grid_latitud

In [9]:
set(ncds.variables.values()) - set(ncds.dimensions.values())

{<class 'netCDF4._netCDF4.Variable'>
 float32 air_pressure_at_sea_level(time, grid_latitude, grid_longitude)
     standard_name: air_pressure_at_sea_level
     units: Pa
     um_stash_source: m01s16i222
     grid_mapping: rotated_latitude_longitude
     coordinates: forecast_period forecast_reference_time
 unlimited dimensions: 
 current shape = (33, 219, 286)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 <class 'netCDF4._netCDF4.Variable'>
 float32 air_temperature(time, grid_latitude, grid_longitude)
     standard_name: air_temperature
     units: K
     um_stash_source: m01s03i236
     grid_mapping: rotated_latitude_longitude
     coordinates: forecast_period forecast_reference_time height
 unlimited dimensions: 
 current shape = (33, 219, 286)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 <class 'netCDF4._netCDF4.Variable'>
 float32 cloud_area_fraction(time_0, grid_latitude, grid_longitude)
     standard_name: cloud_area_fraction
     units: 1
   

In [35]:
dim = list(ncds.dimensions.values())[-2]
dim

<class 'netCDF4._netCDF4.Dimension'>: name = 'time_0', size = 32

In [36]:
import numpy as np

In [37]:
np.diff(ncds[dim.name][()])

masked_array(data=[3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
                   3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
                   3., 3., 3.],
             mask=False,
       fill_value=1e+20)